In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

# שבוע 8 - Numpy מתקדם


שבוע זה ממשיך את הבסיס ממחברת NumPy קודמת, ומעמיקה בנושאים שימושיים לפיסיקה חישובית:
- פעולות מטריציוניות: @, np.matmul, np.dot, transpose, ו-np.linalg.solve.
- ערכים/וקטורים עצמיים: np.linalg.eig, np.linalg.eigvals, קשרים ל-trace ול-det; אינטואיציה פיסיקלית.
- SVD (בלי הוכחות): np.linalg.svd ורעיון דחיסה/פירוק מצבים.
- מחולל אקראי מודרני: np.random.default_rng, seed ושיחזור תוצאות; התפלגויות נפוצות ודגימה וקטורית.
- דוגמא מסכמת: הערכת מונטה-קרלו של π ותנועת דיפוזיה בהליכת-אקראי 1D.
- הערה: בחלק ה-SVD ניתן (אך לא חובה) להסתמך על הדגמה מהרפרנס של NumPy:
https://numpy.org/numpy-tutorials/content/tutorial-svd.html

## ייבוא ספריות והגדרות ראשוניות

נשתמש ב-NumPy וב-Matplotlib. נגדיר גם מחולל אקראי מודרני המאפשר שחזור של קוד על אף השימוש במספרים אקראיים (reproducibility).
מחולל מספרים "אקראיים" במחשב הוא למעשה **פסאודו-אקראי**: הוא מייצר רצף דטרמיניסטי. קביעת **seed** מגדירה את המצב ההתחלתי ולכן מבטיחה יכולת שחזור מלאה של הניסוי:
אותו seed ⇒ אותו רצף ⇒ אותם תוצאות. 
זה קריטי למחקר חישובי: דיבוג, בדיקות יחידה, ביקורות עמיתים, והפעלות חוזרות.

במחברת זאת נשתמש ב`default_rng` ממספר סיבות:
1. **בידוד מצב**:  `default_rng` יוצרת כל קריאה ל `Generator` ששומר מצב פנימי משלו. כך נמנע מיצירה של "מצב גלובלי" כמו ב-`np.random.seed`/`np.random.*` הישנים.
2. **יציבות**: קוד המשתמש ב-`Generator` צפוי יותר לאורך גרסאות, וקל יותר לשלוט בו.
3. **ממשק נקי**: פונקציות דגימה אחידות תחת `rng.*` (למשל `rng.normal`, `rng.poisson`, `rng.choice`).

כיצד נקבע seed?
- נקבע seed בראשית הקוד בפתיחה (למשל `seed=42`) כדי לאפשר רפליקציה.
- נקפיד **לא** לעשות re-seed בלולאה או בכל קריאה ,זה מאתחל את אותו הרצף שוב ושוב (באג נפוץ).
- העדיפו פונקציות שמקבלות `rng` כפרמטר — כך תכתבו קוד נקי מתלות גלובלית.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Modern RNG (prefer over legacy np.random.*)
rng = np.random.default_rng(seed=42)

np.__version__, "RNG OK"

('2.3.2', 'RNG OK')

## פעולות מטריציוניות
### @ ו-np.matmul
האופרטור @ שקול ל-np.matmul עבור מכפלות מטריציות. שניהם תומכים בבאצ'ינג (batch/broadcast) במימדים גבוהים.
שימו לב כי גם האופרטור וגם הפונקציה **לא** מבצעים כפל איבר-איבר; לשם כך משתמשים ב-*.

### np.dot
עבור שני וקטורים – מחזיר מכפלה סקלרית.
עבור מטריצות דו-ממדיות – שקול למכפלת מטריצות.
במימדים גבוהים הכללים שונים מעט (כפל לאורך הצירים האחרונים) ולכן נעדיף @/matmul לקוד ברור.

### transpose / .T
טרנספוז של מטריצה 2D מחליף שורות בעמודות.

In [ ]:
# Basic demo: @ vs dot
A = np.array([[2., 1.],
              [0., 3.]])
B = np.array([[1., -1.],
              [4.,  2.]])

AB1 = A @ B            # matrix product via @
AB2 = np.matmul(A, B)  # same
AB3 = np.dot(A, B)     # same for 2D

A_T = A.T
B_T = B.T

AB1, AB2, np.allclose(AB1, AB2), np.allclose(AB1, AB3), A_T, B_T



באצ'ינג (Batch) ו-Broadcasting במכפלות מטריציות

In [ ]:
# Batched matrix-vector multiply: shape (batch, n, n) @ (n,)
batch = 5
M = rng.normal(size=(batch, 3, 3))
v = rng.normal(size=3)

# Broadcasting: each 3x3 in M multiplies the same vector v -> (batch, 3)
Mv = M @ v

# Batched matrix-matrix: (batch, n, n) @ (batch, n, n) -> (batch, n, n)
N = rng.normal(size=(batch, 3, 3))
MM = M @ N
Mv.shape, MM.shape


## פתרון מערכות לינאריות: `np.linalg.solve`

כאשר נדרש פתרון של מערכת לינארית מהצורה:

\[
A \, x = b,
\]

יש שתי גישות עיקריות:
1. לחשב את \(A^{-1}\) ואז להציב \(x = A^{-1}b\).
2. להשתמש ישירות באלגוריתם נומרי ל-`solve`.

### למה לא לחשב \(A^{-1}\)?
* **חוסר יציבות נומרית**: חישוב מפורש של \(A^{-1}\) עלול להעצים שגיאות עיגול, במיוחד כאשר המטריצה קרובה לסינגולריות (ill-conditioned).
* **יעילות**: מציאת הפתרון באמצעות `np.linalg.solve` משתמשת בפירוק LU (או שיטות דומות) המותאם ישירות לפתרון מערכת, מהיר יותר מחישוב מטריצה הפוכה מלאה.

### דוגמה מתמטית
נניח \(A\) בגודל \(n\times n\).  
- חישוב \(A^{-1}\) דורש \(O(n^3)\) פעולות, ואחר כך כפל מטריציוני \(O(n^2)\).
- לעומת זאת, `np.linalg.solve` פותר את המערכת ב-\(O(n^3)\) **בלי** לייצר מטריצה הפוכה מיותרת.

### הקשר פיסיקלי
מערכות לינאריות \(Ax=b\) צצות בפיסיקה בכל מקום:
* **מעגלים חשמליים**: לפי חוקי אוהם וקירכהוף, מקבלים מערכת משוואות לינאריות בזרמים/מתחים.
* **מכניקה קלאסית**: מערכות של קפיצים/מסות קטנות עם חיבורים לינאריים → נדרשים הפתרונות לכוחות/תזוזות.
* **אנליזת תגובה (response analysis)**: פתרון משוואות לינאריות לתגובה של מערכת בזמן/במרחב.
* **חישוב התפתחות בזמן קצר**: למשל באינטגרציה נומרית של משוואות דיפרנציאליות — צעד אחד כולל פתרון לינארי.

### מדד חשוב: מספר התניה (Condition Number)
אם \(\kappa(A)\) גדול, אז הפתרון רגיש מאוד לשגיאות קטנות ב-\(b\) או ב-\(A\).  
בפועל ניתן לבדוק:
\[
\kappa(A) = \|A\| \, \|A^{-1}\|,
\]
עם `np.linalg.cond(A)`. ערכים גדולים מצביעים על בעיית יציבות.

### כלל אצבע
**אל תחשבו \(A^{-1}\)** אלא השתמשו ב-`np.linalg.solve`, ובדקו את ה-condition number אם אתם חושדים שמדובר במטריצה "כמעט סינגולרית".


In [3]:
# Solve Ax = b (single RHS)
A = np.array([[3., 1.],
              [1., 2.]])
b = np.array([9., 8.])

x = np.linalg.solve(A, b)
residual = A @ x - b
x, residual


(array([2., 3.]), array([0., 0.]))

In [ ]:
# Solve Ax = B (multiple RHS columns)
B = np.column_stack([b, np.array([1., 0.])])  # two RHS vectors
X = np.linalg.solve(A, B)
np.allclose(A @ X, B), X


In [ ]:
# Conditioning matters: nearly singular matrix
C = np.array([[1., 1.],
              [1., 1.000001]])
condC = np.linalg.cond(C)  # condition number
d = np.array([2., 2.000001])
sol = np.linalg.solve(C, d)
condC, sol


מתי להשתמש ב-lstsq? בבעיות least-squares (מערכת עודפת/חסרה), השתמשו ב-np.linalg.lstsq(A, b, rcond=None).

In [ ]:
# Least-squares: fit y ≈ a + b x
xdata = np.linspace(0, 1, 50)
ytrue = 1.5 + 2.0*xdata
yobs  = ytrue + rng.normal(scale=0.1, size=xdata.size)

Phi = np.column_stack([np.ones_like(xdata), xdata])  # design matrix
coef, *_ = np.linalg.lstsq(Phi, yobs, rcond=None)
coef  # intercept ~1.5, slope ~2.0


ערכים עצמיים, וקטורים עצמיים, עקבה ודטרמיננטה

למטריצה A מתקבלים ערכים עצמיים $\lambda_i$ ווקטורים עצמיים $v_i$ כך ש- $A*v_i = \lambda_i*v_i$
בפיסיקה: אלו מצבים נורמליים במערכות לינאריות (לדוגמה, תנודות קטנות של מערכת מסוימת), כיווני יציבות/אי-יציבות, וצירי עיקריים בקירוב קוודרטי.

np.linalg.eig(A) מחזירה (w, V) כאשר w – ערכים עצמיים, ו-V – וקטורים עצמיים בעמודות.

np.linalg.eigvals(A) מחזירה רק ערכים עצמיים.

עבור מטריצה סימטרית (הרמיטית ממשית), הערכים עצמיים ממשיים ווקטורים עצמיים אורתונורמליים.

קשרים שימושיים:
trace(A) = סכום הערכים העצמיים.
det(A) = מכפלת הערכים העצמיים.

In [ ]:
# Symmetric example: prefer eigh
A = np.array([[4., 1., 0.],
              [1., 3., 1.],
              [0., 1., 2.]], dtype=float)

w, V = np.linalg.eigh(A)     # sorted ascending
trA = np.trace(A)
detA = np.linalg.det(A)

trace_vs_sum = trA, np.sum(w)
det_vs_prod   = detA, np.prod(w)

# Check orthonormality and reconstruction A ≈ V diag(w) V^T
Icheck = V.T @ V
Arec = V @ np.diag(w) @ V.T
np.allclose(Icheck, np.eye(3)), np.allclose(Arec, A), trace_vs_sum, det_vs_prod


In [ ]:
# Non-symmetric example: complex eigenvalues are possible
B = np.array([[0., -1.],
              [1.,  0.]])
wB, VB = np.linalg.eig(B)
wB, VB


אזכור: SVD – פירוק לערכים סינגולריים (ללא הוכחות)
ה-SVD מפרק מטריצה  $A$ ל$U\Sigma V^T$, כאשר $\Sigma$ מכילה ערכים סינגולריים אי-שליליים מסודרים מהגדול לקטן. שימושים:

דחיסת מידע/הפחתת מימד -

ניתוח מצבים דומיננטיים -

- least-squares יציב נומרית.

להדגמה נבצע SVD על תמונה (או על מטריצה מלאכותית אם ספריות חיצוניות לא זמינות).

In [ ]:
# Try loading a demo image; fall back to a synthetic image if SciPy is unavailable
try:
    from scipy.datasets import face  # SciPy >= 1.10
    img = face()
except Exception:
    # Simple synthetic "image": gradient + circle
    Y, X = np.ogrid[:256, :256]
    circle = (np.hypot(X-128, Y-128) < 90).astype(float)
    grad = (X / X.max())[None, :]
    img_gray_syn = 0.6*grad + 0.4*circle
    img = np.stack([img_gray_syn, img_gray_syn, img_gray_syn], axis=-1)

# Convert to grayscale using standard luminance weights
img = img.astype(float)
img = img / (img.max() if img.max() > 1 else 255.0)
img_gray = img @ np.array([0.2126, 0.7152, 0.0722])

# SVD
U, s, Vt = np.linalg.svd(img_gray, full_matrices=False)
U.shape, s.shape, Vt.shape, float(s[0]), float(s[-1])


In [ ]:
# Low-rank approximation via top-k singular values
k = 25  # play with k
approx = (U[:, :k] * s[:k]) @ Vt[:k, :]

plt.figure()
plt.title("SVD approximation with k components")
plt.imshow(approx)
plt.axis("off")
plt.show()

# Energy captured by top-k singular values
energy = np.cumsum(s**2) / np.sum(s**2)
energy_k = energy[k-1]
error_rel = np.linalg.norm(img_gray - approx) / np.linalg.norm(img_gray)
energy_k, error_rel


In [ ]:
מחולל אקראי מודרני: np.random.default_rng

לשימוש עדכני ומומלץ באקראיות:

צור מחולל: rng = np.random.default_rng(seed) (סיד קבוע לשחזור).

דגימות נפוצות: rng.uniform, rng.normal, rng.poisson, rng.binomial, rng.choice, ועוד.

וקטוריזציה: דוגמים מערכים גדולים בבת אחת בלי לולאות פייתון.

In [ ]:
# Reproducibility demo: same seed -> same stream
rng1 = np.random.default_rng(123)
rng2 = np.random.default_rng(123)
np.allclose(rng1.normal(size=5), rng2.normal(size=5))


In [ ]:
# Basic distributions
u = rng.uniform(size=10)                      # Uniform [0,1)
z = rng.normal(loc=0.0, scale=1.0, size=10_000)  # Standard normal
p = rng.poisson(lam=3.0, size=10_000)         # Poisson(λ=3)
b = rng.binomial(n=10, p=0.3, size=10_000)    # Binomial
u[:5], z.mean(), z.std(ddof=1), p.mean(), b.mean()


In [ ]:
# Multivariate normal (useful for correlated noise)
mu = np.array([0.0, 0.0])
Sigma = np.array([[1.0, 0.8],
                  [0.8, 1.5]])
X = rng.multivariate_normal(mu, Sigma, size=5_000)
X.shape, np.mean(X, axis=0), np.cov(X.T)


In [ ]:
# Multivariate normal (useful for correlated noise)
mu = np.array([0.0, 0.0])
Sigma = np.array([[1.0, 0.8],
                  [0.8, 1.5]])
X = rng.multivariate_normal(mu, Sigma, size=5_000)
X.shape, np.mean(X, axis=0), np.cov(X.T)


In [ ]:
# Quick histograms (separate figures)
plt.figure()
plt.title("Normal(0,1) – empirical")
plt.hist(z, bins=50, density=True)
plt.xlabel("x"); plt.ylabel("density")
plt.show()

plt.figure()
plt.title("Poisson(λ=3) – empirical")
plt.hist(p, bins=np.arange(p.min(), p.max()+1)-0.5, density=True)
plt.xlabel("k"); plt.ylabel("probability")
plt.show()


תהליך פואסוני – זמני הגעה (Exponential interarrivals)

In [ ]:
# Poisson process via exponential interarrival times
lam = 2.0  # events per unit time
n_events = 2000
inter = rng.exponential(scale=1/lam, size=n_events)  # mean 1/λ
times = np.cumsum(inter)
times[:5], inter.mean()


In [ ]:
# Histogram of interarrival times ~ Exponential(λ)
plt.figure()
plt.title("Poisson process interarrival times")
plt.hist(inter, bins=50, density=True)
plt.xlabel("Δt"); plt.ylabel("density")
plt.show()


## דוגמה מסכמת (1): הערכת מונטה–קרלו של π

הרעיון: לדגום \(N\) נקודות אחידות בריבוע \([-1,1]\times[-1,1]\),  
ולחשב את החלק מהן שנמצא בתוך מעגל היחידה.  

\[
\hat{\pi} \;=\; 4 \cdot \frac{\#\{(x,y): x^2+y^2 \le 1\}}{N}.
\]

בפועל נשתמש במחולל אקראי מודרני ונראה שהשגיאה יורדת בקצב סדר גודל של \(1/\sqrt{N}\).


In [2]:
def mc_pi(N, seed=None):
    # Monte-Carlo estimate of pi using hit/miss in unit disk
    r = np.random.default_rng(seed)
    xy = r.uniform(-1.0, 1.0, size=(N, 2))
    inside = (xy[:,0]**2 + xy[:,1]**2) <= 1.0
    return 4.0 * inside.mean()

Ns = np.logspace(2, 6, num=9, base=10, dtype=int)
est = np.array([mc_pi(int(N), seed=123) for N in Ns])
abs_err = np.abs(est - np.pi)

Ns, est[:5], abs_err[:5]


NameError: name 'np' is not defined

In [ ]:
plt.figure()
plt.title("π estimate vs N (log scale)")
plt.plot(Ns, est, marker='o', label="estimate")
plt.axhline(np.pi, linestyle='--', label="π (true)")
plt.xscale('log')
plt.xlabel("N")
plt.ylabel("estimate")
plt.legend()
plt.show()

plt.figure()
plt.title("|error| vs N (expected ~ 1/√N)")
plt.loglog(Ns, abs_err, marker='o')
plt.xlabel("N")
plt.ylabel("|error|")
plt.show()


## דוגמה מסכמת (2): דיפוזיה בהליכת-אקראי 1D

נבצע \(M\) הליכות אקראיות באורך \(T\) צעדים.  
בכל צעד מתקיים:
\[
x_{t+1} \;=\; x_t + \xi_t, \qquad \xi_t \in \{-1, +1\}, \quad P(\xi_t=+1)=P(\xi_t=-1)=\tfrac{1}{2}.
\]

בממוצע אנליטי מתקבל:
\[
\mathrm{MSD}(t) = \mathbb{E}[\,x_t^2\,] \;\propto\; t.
\]

נראה זאת נומרית, וכן נבדוק את התפלגות נקודת הסיום לאחר \(T\) צעדים — הצפויה להתקרב להתפלגות נורמלית לפי משפט הגבול המרכזי.


In [ ]:
def random_walk_1d(T, M, seed=None):
    # Vectorized ±1 steps and cumulative sums
    r = np.random.default_rng(seed)
    steps = r.choice([-1, 1], size=(M, T))
    x = np.cumsum(steps, axis=1)
    return x

T = 1000
M = 10_000
X = random_walk_1d(T=T, M=M, seed=123)

# Mean-squared displacement over time
msd = (X**2).mean(axis=0)

T, M, msd[:5]


In [ ]:
# MSD ~ t (linear scaling)
t = np.arange(1, T+1)

plt.figure()
plt.title("1D Random Walk – MSD(t) ~ t")
plt.plot(t, msd, label="MSD (empirical)")
# Linear guide normalized to the last point
guide = msd[-1] * (t / t[-1])
plt.plot(t, guide, linestyle='--', label="~linear guide")
plt.xlabel("t")
plt.ylabel("MSD")
plt.legend()
plt.show()


התפלגות נקודת הסיום וה-CLT (התכנסות לנורמלית)

In [ ]:
# Endpoint distribution after T steps should be ~ Normal(0, T)
endpoints = X[:, -1]
mu_hat, var_hat = endpoints.mean(), endpoints.var(ddof=1)

plt.figure()
plt.title("Endpoints histogram ~ N(0, T)")
plt.hist(endpoints, bins=60, density=True)
plt.xlabel("x_T"); plt.ylabel("density")
plt.show()

mu_hat, var_hat, "theory var ~ T =", T
